# CityBikes

In [1]:
# Imports
import requests
import pandas as pd

Send a request to CityBikes for the city of your choice. 

In [4]:
# GET request to the API
get_response = requests.get("http://api.citybik.es/v2/networks")
networks = get_response.json()['networks']

networks

[{'id': 'abu-dhabi-careem-bike',
  'name': 'Abu Dhabi Careem BIKE',
  'location': {'latitude': 24.4866,
   'longitude': 54.3728,
   'city': 'Abu Dhabi',
   'country': 'AE'},
  'href': '/v2/networks/abu-dhabi-careem-bike',
  'company': ['Careem'],
  'gbfs_href': 'https://dubai.publicbikesystem.net/customer/gbfs/v2/en/gbfs.json'},
 {'id': 'acces-velo-saguenay',
  'name': 'Accès Vélo',
  'location': {'latitude': 48.433333,
   'longitude': -71.083333,
   'city': 'Saguenay',
   'country': 'CA'},
  'href': '/v2/networks/acces-velo-saguenay',
  'company': ['PBSC Urban Solutions'],
  'gbfs_href': 'https://saguenay.publicbikesystem.net/customer/gbfs/v2/gbfs.json'},
 {'id': 'aksu',
  'name': 'Aksu',
  'location': {'latitude': 41.1664,
   'longitude': 80.2617,
   'city': '阿克苏市 (Aksu City)',
   'country': 'CN'},
  'href': '/v2/networks/aksu',
  'company': ['阿克苏公共服务']},
 {'id': 'alba',
  'name': 'Alba',
  'location': {'latitude': 44.716667,
   'longitude': 8.083333,
   'city': 'Alba',
   'country':

In [5]:
id = None

# Find the network with the city name "Hamilton"
for network in networks:
    if "Hamilton" in network['location']['city']:
        network_find = network
        break

if not network:
    print("Network not found.")
    exit()

network_id = network['id']
print(f"Network ID: {network_id}")

Network ID: sobi-hamilton


In [6]:
network_id = 'sobi-hamilton'  
url = f"http://api.citybik.es/v2/networks/{network_id}"

response = requests.get(url)

if response.status_code != 200:
    print(f"Failed to retrieve data: {response.status_code}")
    exit()

data = response.json()
stations = data['network']['stations']

In [10]:
# Checking the output
# data
stations

[{'id': '024a3edf037cb411d16acc08a7fcb954',
  'name': 'Bay at Strachan',
  'latitude': 43.26785864558976,
  'longitude': -79.8679232597351,
  'timestamp': '2024-12-13T23:56:46.377183Z',
  'free_bikes': 1,
  'empty_slots': 23,
  'extra': {'uid': 'hub_578',
   'renting': 1,
   'returning': 1,
   'last_updated': 1734130187,
   'address': '325 Bay Street North, Hamilton',
   'payment': ['key',
    'applepay',
    'androidpay',
    'transitcard',
    'accountnumber',
    'phone'],
   'payment-terminal': False}},
 {'id': '0263c2af4dcdc215b9c81753a8df8a9a',
  'name': 'MAC Rack ITB',
  'latitude': 43.25917772795342,
  'longitude': -79.92050260305405,
  'timestamp': '2024-12-13T23:56:46.391463Z',
  'free_bikes': 2,
  'empty_slots': 12,
  'extra': {'uid': 'hub_6255',
   'renting': 1,
   'returning': 1,
   'last_updated': 1734130187,
   'address': '2000 College Court, McMaster University, Hamilton',
   'payment': ['key',
    'applepay',
    'androidpay',
    'transitcard',
    'accountnumber',
  

In [8]:
stations[0].keys()

dict_keys(['id', 'name', 'latitude', 'longitude', 'timestamp', 'free_bikes', 'empty_slots', 'extra'])

In [9]:
stations[0]['extra'].keys()

dict_keys(['uid', 'renting', 'returning', 'last_updated', 'address', 'payment', 'payment-terminal'])

Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [11]:
bike_network = []

# Loop through the stations and extract the information
for station in stations:
    station_info = {
        'station_id': station.get('id'),
        'station_name': station.get('name'),
        'station_address': station.get('extra', {}).get('address'),
        'station_latitude': station.get('latitude'),
        'station_longitude': station.get('longitude'),
        'available_bikes': station.get('free_bikes'),
        'empty_bike_slots': station.get('empty_slots')
    }
    bike_network.append(station_info)

Put your parsed results into a DataFrame.

In [19]:
citybikes = pd.DataFrame(bike_network)

# Checking the output
print(citybikes.shape)
print(citybikes.columns)
print(citybikes.nunique())

(190, 7)
Index(['station_id', 'station_name', 'station_address', 'station_latitude',
       'station_longitude', 'available_bikes', 'empty_bike_slots'],
      dtype='object')
station_id           190
station_name         190
station_address      163
station_latitude     190
station_longitude    190
available_bikes       19
empty_bike_slots      25
dtype: int64


In [20]:
citybikes.head()

,station_id,station_name,station_address,station_latitude,station_longitude,available_bikes,empty_bike_slots
0,024a3edf037cb411d16acc08a7fcb954,Bay at Strachan,"325 Bay Street North, Hamilton",43.267859,-79.867923,1,23
1,0263c2af4dcdc215b9c81753a8df8a9a,MAC Rack ITB,"2000 College Court, McMaster University, Hamilton",43.259178,-79.920503,2,12
2,0265404843dccbf9378ce9d56a570914,Augusta at John,"62-64 Augusta Street, Hamilton",43.251632,-79.868980,0,10
3,05e87b8200ed015e7fb61865b83e3013,Hunter GO Centre,"36 Hunter Street East, Hamilton",43.253330,-79.869533,6,11
4,05fffdca01b4d886a823b025b6a04080,Maple at Rothsay - ERI08,"394 Maple Avenue, Hamilton",43.242163,-79.825003,4,5


In [ ]:
# Save the data to a CSV file
citybikes.to_csv('../data/sql_ready/citybikes.csv', index=False)